<a href="https://colab.research.google.com/github/douglasmasho/MedAlgo/blob/main/TumorGrade2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install monai

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import os
import glob
from sklearn.model_selection import train_test_split

extract_dir = '/content/drive/MyDrive/BRATS'

def find_nifti_files(root_dir, label):
    """Recursively find all .nii files in subdirectories and assign a label."""
    file_paths = []
    for subdir, _, _ in os.walk(root_dir):
        file_paths.extend(glob.glob(os.path.join(subdir, '*.nii')))
    return file_paths, [label] * len(file_paths)

# Define paths
hgg_path = os.path.join(extract_dir, "MICCAI_BraTS_2019_Data_Training",'HGG')
lgg_path = os.path.join(extract_dir, "MICCAI_BraTS_2019_Data_Training",'LGG')

# Find all .nii files
hgg_files, hgg_labels = find_nifti_files(hgg_path, 1)  # High-grade glioma
lgg_files, lgg_labels = find_nifti_files(lgg_path, 0)  # Low-grade glioma

print(f'HGG files: {len(hgg_files)}')
print(f'LGG files: {len(lgg_files)}')

# Combine and split data
all_files = hgg_files + lgg_files
all_labels = hgg_labels + lgg_labels

print(f'All files: {len(all_files)}')
print(f'All labels: {len(all_labels)}')

# Ensure non-empty lists before splitting
if len(all_files) == 0 or len(all_labels) == 0:
    raise ValueError("The file paths or labels are empty. Check the directory and file extensions.")

train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)

In [ ]:
import monai
from monai.transforms import Compose, LoadImage, ScaleIntensity, EnsureChannelFirst, Resize, ToTensor
from monai.data import Dataset, DataLoader

# Define transformations
transforms = Compose([
    LoadImage(image_only=True),  # Load NIfTI image
    ScaleIntensity(),            # Normalize intensity
    EnsureChannelFirst(),        # Add channel dimension
    Resize((128, 128, 128)),     # Resize to a fixed size
    ToTensor()                   # Convert to tensor
])

# Create custom dataset class
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = image_path  # Load image directly
        if self.transform:
            image = self.transform(image)
        return {'image': image, 'label': label}

# Create datasets and dataloaders
train_dataset = CustomDataset(train_files, train_labels, transform=transforms)
val_dataset = CustomDataset(val_files, val_labels, transform=transforms)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=2)


In [ ]:
!pip install monai


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from monai.networks.nets import DenseNet121
from monai.transforms import Compose, LoadImage, ScaleIntensity, EnsureChannelFirst, Resize, RandAffine, RandRotate, RandZoom, ToTensor
from monai.data import Dataset, DataLoader
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
import nibabel as nib

extract_dir = '/content/drive/MyDrive/BRATS'

# Define paths
hgg_path = os.path.join(extract_dir, "MICCAI_BraTS_2019_Data_Training", 'HGG')
lgg_path = os.path.join(extract_dir, "MICCAI_BraTS_2019_Data_Training", 'LGG')

# Define a function to find all .nii files within each subject's directory
def find_nifti_files(root_dir, label):
    subject_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    file_paths = []
    for subject_dir in subject_dirs:
        nii_files = glob.glob(os.path.join(subject_dir, '*.nii'))
        if nii_files:
            file_paths.append(nii_files[0])  # Taking the first .nii file as an example
    return file_paths, [label] * len(file_paths)

# Find all .nii files
hgg_files, hgg_labels = find_nifti_files(hgg_path, 1)
lgg_files, lgg_labels = find_nifti_files(lgg_path, 0)

all_files = hgg_files + lgg_files
all_labels = hgg_labels + lgg_labels

# Convert lists to numpy arrays for resampling
all_files_np = np.array(all_files)
all_labels_np = np.array(all_labels)

# Oversample the minority class
ros = RandomOverSampler(random_state=42)
resampled_files, resampled_labels = ros.fit_resample(all_files_np.reshape(-1, 1), all_labels_np)
resampled_files = resampled_files.flatten()

# Split resampled data
train_files, val_files, train_labels, val_labels = train_test_split(
    resampled_files, resampled_labels, test_size=0.2, stratify=resampled_labels, random_state=42
)

# Define transformations with augmentation
transforms = Compose([
    ScaleIntensity(),
    Resize((128, 128)),
    RandAffine(prob=0.5),
    RandRotate(range_x=(0, 15), prob=0.5),
    RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
    ToTensor()
])

# Create custom dataset class for 2D slices
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None, augment_minority=False):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.augment_minority = augment_minority

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load the NIfTI file
        img = nib.load(image_path).get_fdata()

        # Choose a random slice along the axial plane (e.g., middle slice)
        slice_index = img.shape[2] // 2
        image_slice = img[:, :, slice_index]

        # Add channel dimension for grayscale
        image_slice = np.expand_dims(image_slice, axis=0)

        if self.augment_minority and label == 0:
            if self.transform:
                image_slice = self.transform(image_slice)
        else:
            if self.transform:
                image_slice = self.transform(image_slice)

        return {'image': image_slice, 'label': label}

# Create datasets and dataloaders
batch_size = 4
train_dataset = CustomDataset(train_files, train_labels, transform=transforms, augment_minority=True)
val_dataset = CustomDataset(val_files, val_labels, transform=transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=2)

# Define the model for 2D input
model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=2)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_function = torch.nn.CrossEntropyLoss()

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weight_dict = dict(enumerate(class_weights))

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for batch in train_loader:
        images = batch['image'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Validation step
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            outputs = model(images)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = val_correct_predictions / val_total_predictions
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_accuracy:.4f}')

print('Training complete!')


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1, Loss: 0.5875, Accuracy: 0.6763
Validation Loss: 0.5327, Validation Accuracy: 0.7692
Epoch 2, Loss: 0.5655, Accuracy: 0.7198
Validation Loss: 0.6284, Validation Accuracy: 0.7500
Epoch 3, Loss: 0.5845, Accuracy: 0.6884
Validation Loss: 0.5423, Validation Accuracy: 0.6827
Epoch 4, Loss: 0.5300, Accuracy: 0.7246
Validation Loss: 0.4841, Validation Accuracy: 0.7596
Epoch 5, Loss: 0.5337, Accuracy: 0.7053
Validation Loss: 0.5626, Validation Accuracy: 0.7308
Epoch 6, Loss: 0.5423, Accuracy: 0.7464
Validation Loss: 0.5111, Validation Accuracy: 0.7404
Epoch 7, Loss: 0.5431, Accuracy: 0.7343
Validation Loss: 0.4473, Validation Accuracy: 0.7885
Epoch 8, Loss: 0.5077, Accuracy: 0.7536
Validation Loss: 0.5631, Validation Accuracy: 0.6538
Epoch 9, Loss: 0.4967, Accuracy: 0.7657
Validation Loss: 0.4870, Validation Accuracy: 0.7981
Epoch 10, Loss: 0.5184, Accuracy: 0.7367
Validation Loss: 0.4671, Validation Accuracy: 0.7885
Epoch 11, Loss: 0.5115, Accuracy: 0.7391
Validation Loss: 0.4510, Val